In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import os

import tensorflow as tf
from im2txt import configuration
from im2txt import inference_wrapper
from im2txt.inference_utils import caption_generator
from im2txt.inference_utils import vocabulary
import matplotlib.pyplot as plt
from PIL import Image

import numpy as np
import cv2

import pandas as pd

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
# 训练好的模型存放路径
checkpoint_path = "./model/model.ckpt-3000000"
# 词汇表
vocab_file = "./im2txt/data/word_counts.txt"
# 视频路径
input_files = os.path.join(os.path.dirname(os.getcwd()),"videos/")
# 输出路径
output_files=os.path.join(os.path.dirname(os.getcwd()),"videos_result.csv")

In [3]:
# 储存视频的内容
df = pd.DataFrame(columns=["Video_Path","Video_Content"])

#print(df)

In [4]:
# 载入训练好的模型
g = tf.Graph()
with g.as_default():
    model = inference_wrapper.InferenceWrapper()
    restore_fn = model.build_graph_from_config(configuration.ModelConfig(), checkpoint_path)

# 载入词表
vocab = vocabulary.Vocabulary(vocab_file)


INFO:tensorflow:Building model.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecordDataset`.





Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use `tf.cast` instead.



Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



INFO:tensorflow:Initializing vocabulary from file: ./im2txt/data/word_counts.txt

INFO:tensorflow:Created vocabulary with 11520 words


In [20]:
#进行处理（简单抽帧处理）
with tf.Session(graph=g) as sess:
    # 载入训练好的模型
    restore_fn(sess)
    generator = caption_generator.CaptionGenerator(model, vocab)
    
    #循环文件夹
    for root,dirs,files in os.walk(input_files):
        for file in files:
            
            tmp_content = ''
            
            # 获取视频路径及名称
            video_path = os.path.join(root,file)
    
            # 从文件读取视频内容
            cap = cv2.VideoCapture(video_path)
            # 视频每秒传输帧数
            fps = cap.get(cv2.CAP_PROP_FPS)
            # 视频图像的宽度
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            # 视频图像的长度
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

            fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
            #out = cv2.VideoWriter('video/output.avi',fourcc,fps,(frame_width,frame_height))
            n = 0
            sentence = ''
            while(True):
                # ret 读取成功True或失败False
                # frame读取到的图像的内容
                # 读取一帧数据
                ret,frame = cap.read()
                if ret!=True:
                    break
                n += 1
                # 每秒生成一个新的描述
                if n >= fps:
                    n = 0
                    cv2.imwrite('temp.jpg', frame)
                    # 载入图片
                    image = tf.gfile.FastGFile('temp.jpg', 'rb').read()   
                    # 获得图片描述
                    captions = generator.beam_search(sess, image)
                    # 获得图片描述
                    sentence = [vocab.id_to_word(w) for w in captions[0].sentence[1:-1]]
                    sentence = " ".join(sentence)
                    
                    #将图片描述保存到临时变量中
                    tmp_content = tmp_content + sentence
                    #print(tmp_content)
            
                # cv2.putText(图像, 文字, (x, y), 字体, 大小, (b, g, r), 宽度)
                #frame = cv2.putText(frame, sentence, (50, frame_height-50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                # 写入frame
                #out.write(frame)
        
            #将图片路径以及内容保存
            tmp_df = pd.DataFrame(data=[[video_path,tmp_content]], columns=["Video_Path","Video_Content"])
            df = df.append(tmp_df)
            
            print(file+" Success!")

cap.release()
cv2.destroyAllWindows()
print("Process Success!")

INFO:tensorflow:Loading model from checkpoint: ./model/model.ckpt-3000000
INFO:tensorflow:Restoring parameters from ./model/model.ckpt-3000000
INFO:tensorflow:Successfully loaded checkpoint: model.ckpt-3000000
1.mp4 Success!
2.mp4 Success!
Process Success!


In [7]:
#进行处理（关键帧处理）
import ShortDetect
from ShortDetect import shot_detector

In [8]:
with tf.Session(graph=g) as sess:
    # 载入训练好的模型
    restore_fn(sess)
    generator = caption_generator.CaptionGenerator(model, vocab)
    
    #循环文件夹
    for root,dirs,files in os.walk(input_files):
        for file in files:
            
            tmp_content = ''
            
            # 获取视频路径及名称
            video_path = os.path.join(root,file)
            
            #关键帧提取
            detector = shot_detector(video_path, output_dir='./', thres=1.5)
            detector.run()
            #print(detector.shots)
    
            # 从文件读取视频内容
            cap = cv2.VideoCapture(video_path)
            # 视频图像的宽度
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            # 视频图像的长度
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

            fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
            #out = cv2.VideoWriter('video/output.avi',fourcc,fps,(frame_width,frame_height))
            n = 0
            sentence = ''
            
            for n,number in detector.shots:
                # ret 读取成功True或失败False
                # frame读取到的图像的内容
                # 读取一帧数据
                cap.set(cv2.CAP_PROP_POS_FRAMES,number - 1)  #设置要获取的帧号，这里取number-1有待商榷
                ret,frame = cap.read()
                cv2.imwrite('temp.jpg', frame)
                # 载入图片
                image = tf.gfile.FastGFile('temp.jpg', 'rb').read()   
                # 获得图片描述
                captions = generator.beam_search(sess, image)
                # 获得图片描述
                sentence = [vocab.id_to_word(w) for w in captions[0].sentence[1:-1]]
                sentence = " ".join(sentence)
                    
                #将图片描述保存到临时变量中
                tmp_content = tmp_content + sentence
                #print(tmp_content)
            
                # cv2.putText(图像, 文字, (x, y), 字体, 大小, (b, g, r), 宽度)
                #frame = cv2.putText(frame, sentence, (50, frame_height-50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                # 写入frame
                #out.write(frame)
        
            #将图片路径以及内容保存
            tmp_df = pd.DataFrame(data=[[video_path,tmp_content]], columns=["Video_Path","Video_Content"])
            df = df.append(tmp_df)
            
            print(file+" Success!")

cap.release()
cv2.destroyAllWindows()
print("Process Success!")

INFO:tensorflow:Loading model from checkpoint: ./model/model.ckpt-3000000
INFO:tensorflow:Restoring parameters from ./model/model.ckpt-3000000
INFO:tensorflow:Successfully loaded checkpoint: model.ckpt-3000000
max diff: 0.4407269 min diff: 0.030927297
Instructions for updating:
Use tf.gfile.GFile.
1.mp4 Success!
max diff: 0.42285776 min diff: 0.034160096
2.mp4 Success!
Process Success!


In [9]:
#将结果储存到csv中
df.reset_index(drop=True, inplace=True)
df.to_csv(output_files)